In [84]:
import pickle
import sys
import os
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
# Dependencies
import pickle
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [161]:
reqdf = pd.read_csv(r'C:\Users\johd003\Documents\GitHub\data\reqdata.csv')

C:\Users\johd003\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (4,15,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [171]:
topn_titles = reqdf['Title'].value_counts().nlargest(15).index.astype(list)
topn_titles

Index(['Administrative Assistant', 'Project Manager', 'Assembly Worker',
       'Business/System Analyst', 'Developer', 'Data Entry Operator',
       'Systems/Software Engineer', 'IT Analyst', 'Recruiter',
       'Graphic Designer', 'Business Analyst', 'Manufacturing Technician',
       'Financial Analyst', 'Sales Manager', 'Design Validation Engineer'],
      dtype='object')

In [176]:
reqdf_topn_titles = reqdf[reqdf['Title'].isin(topn_titles)]
reqdf_topn_titles['Title'].head()

5                     Developer
6              Business Analyst
8                     Developer
23              Project Manager
44    Systems/Software Engineer
Name: Title, dtype: object

In [180]:
df = reqdf_topn_titles[['Title','Description']].dropna().drop_duplicates()
df.head()

,Title,Description
5,Developer,NTE $100 \n\n\n\nJOB DESCRIPTION:\n\tTHE DATA ...
6,Business Analyst,JOB DESCRIPTION:\n\tTHE DATA PRACTICE WEB DEVE...
8,Developer,DESCRIPTION: WORK AS PART OF A BUSINESS SYSTE...
23,Project Manager,MUST SPEAK VERY GOOD ENGLISH \n\nADDITIONAL SK...
44,Systems/Software Engineer,IT SAP ABAP/JAVA DEVELOPER WITH EXPERIENCE IMP...


In [181]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import nltk
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

ps = nltk.PorterStemmer()
#data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
#data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopWords]
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johd003\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [218]:
df['jd_prepped'] = df['Description'].apply(clean_text)

In [219]:
df['jd_prepped'].head()

5     [nte, 100, job, descript, data, qualiti, dq, v...
6     [job, descript, data, practic, web, develop, a...
8     [descript, work, part, busi, system, support, ...
23    [must, speak, good, english, addit, skill, vrf...
44    [sap, abapjava, develop, experi, implement, so...
Name: jd_prepped, dtype: object

In [220]:
df.jd_prepped.iloc[0]

['nte',
 '100',
 'job',
 'descript',
 'data',
 'qualiti',
 'dq',
 'visual',
 'expert',
 'contribut',
 'strateg',
 'function',
 'implement',
 'dq',
 'corpor',
 'involv',
 'work',
 'busi',
 'stakehold',
 'understand',
 'busi']

In [233]:
all_words = pd.Series(df.jd_prepped.iloc[0])
for i in range(1,df.shape[0]):
    s = pd.Series(df.jd_prepped.iloc[i])
    all_words = all_words.append(s,ignore_index=True)

In [234]:
len(all_words)

367457

In [238]:
from collections import defaultdict
import pandas as pd
#text_list = ['france', 'spain', 'spain beaches', 'france beaches', 'spain best beaches']

word_freq = defaultdict(int)

for text in all_words:
    for word in text.split():
        word_freq[word] += 1

word_freq_df = pd.DataFrame.from_dict(word_freq, orient='index') \
.sort_values(0, ascending=False) \
.rename(columns={0: 'abs_freq'})

word_freq_df.head()

,abs_freq
â,8129
manag,4917
project,3949
work,3873
requir,3383


In [239]:
df.head()

,Title,Description,jd_prepped
5,Developer,NTE $100 \n\n\n\nJOB DESCRIPTION:\n\tTHE DATA ...,"[nte, 100, job, descript, data, qualiti, dq, v..."
6,Business Analyst,JOB DESCRIPTION:\n\tTHE DATA PRACTICE WEB DEVE...,"[job, descript, data, practic, web, develop, a..."
8,Developer,DESCRIPTION: WORK AS PART OF A BUSINESS SYSTE...,"[descript, work, part, busi, system, support, ..."
23,Project Manager,MUST SPEAK VERY GOOD ENGLISH \n\nADDITIONAL SK...,"[must, speak, good, english, addit, skill, vrf..."
44,Systems/Software Engineer,IT SAP ABAP/JAVA DEVELOPER WITH EXPERIENCE IMP...,"[sap, abapjava, develop, experi, implement, so..."


In [265]:
def clean_text_no_stem(text):
    # split into words
    lc = text.lower()
    tokens = word_tokenize(lc)
    # remove all tokens that are not alphabetic
    words = [word for word in tokens if word.isalpha()]
    words = [word for word in words if word not in stopWords]
    return words

In [266]:
df['jd_tokened'] = df['Description'].apply(clean_text_no_stem)
df['jd_tokened'].head(15)

5      [nte, job, description, data, quality, dq, vis...
6      [job, description, data, practice, web, develo...
8      [description, work, part, business, system, su...
23     [must, speak, good, english, additional, skill...
44     [sap, developer, experience, implementing, sol...
45     [test, lead, testing, enterprise, services, te...
46     [job, responsibilities, reset, passwords, user...
49     [job, description, enterprise, monitoring, res...
54     [supprot, role, analysts, part, global, team, ...
55     [work, various, internal, groups, backend, con...
77     [manages, one, projects, within, constraints, ...
105    [capable, test, preparation, test, data, manag...
107                        [stabilization, uat, support]
121    [data, interview, ot, responsibilities, estima...
130    [assume, overall, qa, responsibility, along, o...
Name: jd_tokened, dtype: object

In [272]:
    all_words = pd.Series(df.jd_tokened.iloc[0])
for i in range(1,df.shape[0]):
    s = pd.Series(df.jd_tokened.iloc[i])
    all_words = all_words.append(s,ignore_index=True)

In [283]:
word_freq = defaultdict(int)

for text in all_words:
    for word in text.split():
        word_freq[word] += 1

word_freq_df = pd.DataFrame.from_dict(word_freq, orient='index') \
.sort_values(0, ascending=False) \
.rename(columns={0: 'abs_freq'})

word_freq_df.head(5)

,abs_freq
project,2791
work,2676
experience,2531
business,2310
support,2300


In [291]:
word_freq_df['Title'] = 'Top 15'
word_freq_df['Observations'] = df.shape[0]
word_freq_df['Word'] = word_freq_df.index
word_freq_df['Rank'] = word_freq_df.abs_freq.rank(method='dense',ascending=False)
word_freq_df = word_freq_df[word_freq_df['Rank'] <= 25]

,abs_freq,Title,Observations,Word,Rank
project,2791,Top 15,4806,project,1.0
work,2676,Top 15,4806,work,2.0
experience,2531,Top 15,4806,experience,3.0
business,2310,Top 15,4806,business,4.0
support,2300,Top 15,4806,support,5.0
team,1990,Top 15,4806,team,6.0
management,1972,Top 15,4806,management,7.0
job,1782,Top 15,4806,job,8.0
skills,1686,Top 15,4806,skills,9.0
data,1541,Top 15,4806,data,10.0


## Loop through each Title and build a dataframe, printing the top 25 words for each title

In [350]:
wfd = word_freq_df
for i in topn_titles:
    pTitle = i
    dff = df[df['Title'] == pTitle]
    print(dff.shape)

    words = pd.Series(dff.jd_tokened.iloc[0])
    for j in range(1,dff.shape[0]):
        s = pd.Series(dff.jd_tokened.iloc[j])
        words = words.append(s,ignore_index=True)

    word_freq_f = defaultdict(int)
    for text in words:
        for word in text.split():
            word_freq_f[word] += 1


    word_freq_dff = pd.DataFrame.from_dict(word_freq_f, orient='index') \
    .rename(columns={0: 'abs_freq'}) \
    .sort_values('abs_freq', ascending=False)

    word_freq_dff = word_freq_dff.head(25)
    word_freq_dff['Title'] = pTitle
    word_freq_dff['Observations'] = dff.shape[0]
    word_freq_dff['Word'] = word_freq_df.index
    word_freq_dff['Rank'] = word_freq_df.abs_freq.rank(method='dense',ascending=False)
    wfd = wfd.append(word_freq_dff, ignore_index=True)

(837, 4)
(959, 4)
(301, 4)
(409, 4)
(275, 4)
(260, 4)
(317, 4)
(210, 4)
(140, 4)
(66, 4)
(344, 4)
(270, 4)
(315, 4)
(63, 4)
(40, 4)


In [351]:
wfd.shape

(400, 5)

In [324]:
topn_titles

Index(['Administrative Assistant', 'Project Manager', 'Assembly Worker',
       'Business/System Analyst', 'Developer', 'Data Entry Operator',
       'Systems/Software Engineer', 'IT Analyst', 'Recruiter',
       'Graphic Designer', 'Business Analyst', 'Manufacturing Technician',
       'Financial Analyst', 'Sales Manager', 'Design Validation Engineer'],
      dtype='object')

In [355]:
wfd.to_csv(r'C:\Users\johd003\Documents\GitHub\data\req_description_word_freq.csv',index=False)